## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSBSSSBS'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1651
+-------------------+-------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+-------------------+-----------+-----------+------+--------------------+------+
|            coeff_1|            coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|           res_mean|    station|  total_var|undefs|              vector|  year|
+-------------------+-------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+-------------------+-----------+-----------+------+--------------------+------+
| 392.42601213666205| -90.28280871726807|-173.86119095642712|     57.9|BSBSSSBS| 28.3378|   -82.26|       TOBS|0.7880303361777318|0.7768109716183055|0.7352042557241117|0.03216197458

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [24]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


station    year     coeff_3
0  USC00087851  2003.0 -173.861191
1  USC00083316  1914.0  252.298172
2  USC00083321  1963.0 -260.570233
3  USC00081046  1962.0 -130.294761
4  USC00087440  1915.0 -220.545752

In [25]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.head(10)

station  USC00080535  USC00080598  USC00080940  USC00080945  USC00081046  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN   -26.245726   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN   245.448780   
1908.0           NaN          NaN          NaN          NaN   111.230745   
1909.0           NaN          NaN          NaN          NaN    23.210313   
1910.0           NaN          NaN          NaN          NaN   -37.307076   

station  USC00081163  USC00081310  USC00081432  USC00081632  USC00081635  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN    69.835572          NaN          NaN   
1909.0           NaN          NaN   136.201748          NaN          NaN   
1910.0           NaN          NaN    64.316581          NaN          NaN   

station     ...       USC00088530  USC00088565  USC00088573  USC00088782  \
year        ...                                                            
1901.0      ...               NaN          NaN          NaN          NaN   
1902.0      ...               NaN          NaN          NaN          NaN   
1903.0      ...               NaN          NaN  -147.164937          NaN   
1904.0      ...               NaN          NaN  -115.528586          NaN   
1905.0      ...               NaN          NaN   -84.329923          NaN   
1906.0      ...               NaN          NaN   325.477586          NaN   
1907.0      ...               NaN          NaN          NaN          NaN   
1908.0      ...               NaN          NaN          NaN          NaN   
1909.0      ...               NaN          NaN          NaN          NaN   
1910.0      ...               NaN          NaN          NaN          NaN   

station  USC00088824  USC00088890  USC00089120  USC00089176  USC00089430  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0     25.221196          NaN          NaN          NaN          NaN   
1903.0   -110.773226          NaN          NaN          NaN          NaN   
1904.0    -16.231932          NaN          NaN          NaN          NaN   
1905.0     32.919066          NaN          NaN          NaN          NaN   
1906.0    162.964327          NaN          NaN          NaN          NaN   
1907.0     83.298427          NaN          NaN          NaN          NaN   
1908.0     82.433612          NaN          NaN          NaN          NaN   
1909.0     15.687889          NaN          NaN          NaN          NaN   
1910.0     91.811357          NaN          NaN          NaN          NaN   

station  USW00092806  
year                  
1901.0           NaN  
1902.0           NaN  
1903.0           NaN  
1904.0           NaN  
1905.0           NaN  
1906.0           NaN  
1907.0           NaN  
1908.0           NaN  
1909.0           NaN  
1910.0           NaN  

[10 rows x 49 columns]

In [26]:
def RMS(Mat):
    return np.nanmean(Mat**2)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',RMS(year_station_table)
print 'MS removing mean-by-station= ',RMS(tbl_minus_station)
print 'MS removing mean-by-year   = ',RMS(tbl_minus_year)

total MS                   =  28415.5317557
MS removing mean-by-station=  22994.3261225
MS removing mean-by-year   =  15364.8806355


In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station  USC00080535  USC00080598  USC00080940  USC00080945  USC00081046  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN    77.498878   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN   143.068825   
1908.0           NaN          NaN          NaN          NaN    12.644568   
1909.0           NaN          NaN          NaN          NaN   200.266279   
1910.0           NaN          NaN          NaN          NaN   164.661356   

station  USC00081163  USC00081310  USC00081432  USC00081632  USC00081635  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN  -290.868228          NaN          NaN   
1909.0           NaN          NaN   331.978949          NaN          NaN   
1910.0           NaN          NaN  -245.970158          NaN          NaN   

station     ...       USC00088530  USC00088565  USC00088573  USC00088782  \
year        ...                                                            
1901.0      ...               NaN          NaN          NaN          NaN   
1902.0      ...               NaN          NaN          NaN          NaN   
1903.0      ...               NaN          NaN  -262.711629          NaN   
1904.0      ...               NaN          NaN  -169.912089          NaN   
1905.0      ...               NaN          NaN    69.690732          NaN   
1906.0      ...               NaN          NaN  -421.873283          NaN   
1907.0      ...               NaN          NaN          NaN          NaN   
1908.0      ...               NaN          NaN          NaN          NaN   
1909.0      ...               NaN          NaN          NaN          NaN   
1910.0      ...               NaN          NaN          NaN          NaN   

station  USC00088824  USC00088890  USC00089120  USC00089176  USC00089430  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0   -325.353220          NaN          NaN          NaN          NaN   
1903.0    198.699182          NaN          NaN          NaN          NaN   
1904.0     -0.087055          NaN          NaN          NaN          NaN   
1905.0    -89.963924          NaN          NaN          NaN          NaN   
1906.0    -57.454914          NaN          NaN          NaN          NaN   
1907.0    156.800344          NaN          NaN          NaN          NaN   
1908.0     12.030125          NaN          NaN          NaN          NaN   
1909.0    151.081881          NaN          NaN          NaN          NaN   
1910.0    531.418800          NaN          NaN          NaN          NaN   

station  USW00092806  
year                  
1901.0           NaN  
1902.0           NaN  
1903.0           NaN  
1904.0           NaN  
1905.0           NaN  
1906.0           NaN  
1907.0           NaN  
1908.0           NaN  
1909.0           NaN  
1910.0           NaN  

[10 rows x 49 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  195.727025297
RMS removing mean-by-station=  183.306915055
RMS removing mean-by-year   =  131.31596864


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 195.727025297
0 after removing mean by year    = 131.31596864
0 after removing mean by stations= 116.264223899
1 after removing mean by year    = 115.679654512
1 after removing mean by stations= 115.539390957
2 after removing mean by year    = 115.48765606
2 after removing mean by stations= 115.465982349
3 after removing mean by year    = 115.456414624
3 after removing mean by stations= 115.45208948
4 after removing mean by year    = 115.450111848
4 after removing mean by stations= 115.449202429


In [14]:
year_station_table.keys()

Index([u'USC00080535', u'USC00080598', u'USC00080940', u'USC00080945',
       u'USC00081046', u'USC00081163', u'USC00081310', u'USC00081432',
       u'USC00081632', u'USC00081635', u'USC00082008', u'USC00082418',
       u'USC00083153', u'USC00083316', u'USC00083321', u'USC00083322',
       u'USC00083956', u'USC00084289', u'USC00084723', u'USC00085539',
       u'USC00085622', u'USC00085690', u'USC00085935', u'USC00086065',
       u'USC00086414', u'USC00086419', u'USC00086500', u'USC00086880',
       u'USC00087137', u'USC00087205', u'USC00087395', u'USC00087440',
       u'USC00087665', u'USC00087851', u'USC00088021', u'USC00088024',
       u'USC00088076', u'USC00088527', u'USC00088529', u'USC00088530',
       u'USC00088565', u'USC00088573', u'USC00088782', u'USC00088824',
       u'USC00088890', u'USC00089120', u'USC00089176', u'USC00089430',
       u'USW00092806'],
      dtype='object', name=u'station')